In [1]:
from utils import set_device, dino_processor, setup_dino_model#, calculate_embeddings

In [32]:
import pickle
import platform
import os
if platform.system() == 'Darwin':
    DATA_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Data.nosync"
    ROOT_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Thesis"
elif platform.system() == 'Linux':
    DATA_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync"
    ROOT_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis"

current_wd = os.getcwd()

In [15]:
import torch
import os
from glob import glob
from tqdm import tqdm
import numpy as np

In [3]:
device = set_device()
dino_model = setup_dino_model(device)

Using cpu as device


In [33]:
def setup_generator():
    os.chdir(f"{ROOT_PATH}/stylegan2-ada-pytorch")
    # Load model architecture
    experiment_path = f"{DATA_PATH}/Models/Stylegan2_Ada/Experiments/00005-stylegan2_ada_images-mirror-auto2-kimg5000-resumeffhq512/"
    model_name = "network-snapshot-001200.pkl"
    model_path = experiment_path + model_name
    with open(model_path, 'rb') as f:
        architecture = pickle.load(f)
        G = architecture['G_ema']
        D = architecture['D']
    os.chdir(current_wd)
    return G

G = setup_generator()

In [29]:
def get_embedding(input):
    # Load Image and preprocess
    input = dino_processor(input)
    input = input.to(device)
    # Perform forward pass
    with torch.no_grad():
        output = dino_model(input)
        embedding = output['pooler_output']
    return embedding


def calculate_embeddings(source, save_path, generator=None):
    if os.path.exists(save_path):
        print(f"Embeddings for {input_images_path} already calculated")
        return
    
    if isinstance(source, str):
        input_images = glob(f"{source}*.jpg")
        skus = [elem.split('/')[-1].split('.')[0] for elem in input_images]

        embeddings = {elem:None for elem in skus}

        for image_path in tqdm(input_images):
            sku = image_path.split('/')[-1].split('.')[0]
            embedding = get_embedding(image_path)
            embeddings[sku] = embedding.detach().cpu()

    elif isinstance(source, dict):
        if generator == None: 
            raise ValueError("Generator must be defined when using latents as source")
        

        ## Loop over all dict keys, generate image from latent, normalize and clamp image, calculate embedding, assign to embeddings dict

    else:
        raise ValueError(f"Input must be either a path (str) to an image directory or dict with latents")

        
    # Save embeddings
    torch.save(embeddings, save_path)

In [22]:
latents = torch.load("/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Models/e4e/00005_snapshot_1200/inversions/latents_dict.pt")
type(latents)

dict

In [34]:
calculate_embeddings(latents, 't.pkl', G)

UnboundLocalError: local variable 'embeddings' referenced before assignment

### Dino Custom Processor and Model

In [4]:
dino_model = setup_dino_model(device)

### Calculate all embeddings:

In [5]:
# Real Images
input_images_path = f"{DATA_PATH}/Zalando_Germany_Dataset/dresses/images/e4e_images/all/"
save_path = f"{DATA_PATH}/Models/Assessor/DinoV2/Embeddings/real_images_embeddings.pt"
calculate_embeddings(input_images_path, save_path)

# e4e 00003
input_images_path = f"{DATA_PATH}/Generated_Images/e4e/00003_snapshot_920/"
save_path = f"{DATA_PATH}/Models/Assessor/DinoV2/Embeddings/e4e_00003_snapshot_920.pt"
calculate_embeddings(input_images_path, save_path)

# e4e 00005
input_images_path = f"{DATA_PATH}/Generated_Images/e4e/00005_snapshot_1200/"
save_path = f"{DATA_PATH}/Models/Assessor/DinoV2/Embeddings/e4e_00005_snapshot_1200.pt"
calculate_embeddings(input_images_path, save_path)

# PTI
input_images_path = f"{DATA_PATH}/Generated_Images/PTI/"
save_path = f"{DATA_PATH}/Models/Assessor/DinoV2/Embeddings/PTI.pt"
calculate_embeddings(input_images_path, save_path)


# Restyle
input_images_path = f"{DATA_PATH}/Generated_Images/restyle/inference_results/4/"
save_path = f"{DATA_PATH}/Models/Assessor/DinoV2/Embeddings/restyle.pt"
calculate_embeddings(input_images_path, save_path)

Embeddings for /pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Zalando_Germany_Dataset/dresses/images/e4e_images/all/ already calculated
Embeddings for /pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Generated_Images/e4e/00003_snapshot_920/ already calculated
Embeddings for /pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Generated_Images/e4e/00005_snapshot_1200/ already calculated
Embeddings for /pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Generated_Images/PTI/ already calculated
Embeddings for /pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Generated_Images/restyle/inference_results/4/ already calculated
